In [421]:
using DataFrames;
using Distributions;

In [422]:
df = readtable("/Users/nwelch/prelim/data/plantDataTest.csv");
dxixj = convert(Matrix, readtable("/Users/nwelch/prelim/data/dxixjTest.csv"));

In [423]:
dxixj[1:5, 1:5].*dxixj[1:5, 1:5]

5×5 Array{Float64,2}:
  0.0    1.0   36.0   56.25  64.0 
  1.0    0.0   25.0   42.25  49.0 
 36.0   25.0    0.0    2.25   4.0 
 56.25  42.25   2.25   0.0    0.25
 64.0   49.0    4.0    0.25   0.0 

# Open Issues: 
* Note that the last log sums in llRatio_tau have reversed signs than what is shown in the appendix. The Appendix appears to have the wrong sign. 

# Initial Parameter Values

In [424]:
#priors
mu_a=0.7; mu_b=0.004;
theta_a=0.8; theta_b=10;
sigma_a=0.5; sigma_b=1;

# initial/test values
u = 175
th = 1;
s = 10;
t = [2., 10., 14., 20.];
tl = 15.;

# Function Definitions

In [425]:
function getThetafxixj(theta, sigma, dxixj)
    dstSq = dxixj.*dxixj
    twoSigmaSq = 2*(sigma^2)
    thetafxixj = (theta/(twoSigmaSq*pi))*exp(-dstSq/twoSigmaSq)
    thetafxixj
end

getThetafxixj (generic function with 1 method)

In [426]:
thetafxixj = getThetafxixj(th, s, dxixj)

165×165 Array{Float64,2}:
 0.00159155   0.00158361   0.00132937  …  0.000700093  0.00061783 
 0.00158361   0.00159155   0.00140454     0.000785416  0.000700093
 0.00132937   0.00140454   0.00159155     0.00120136   0.00112576 
 0.00120136   0.00128847   0.00157374     0.0012998    0.00123641 
 0.0011557    0.00124571   0.00156003     0.00132771   0.00126929 
 0.00106153   0.0011557    0.00152152  …  0.00137501   0.00132771 
 0.00101355   0.001109     0.00149699     0.00139404   0.00135284 
 0.000869104  0.000965324  0.00140454     0.00143112   0.00140981 
 0.000821567  0.000917097  0.00136815     0.0014365    0.0014222  
 0.000728664  0.000821567  0.00128847     0.0014365    0.0014365  
 0.000683662  0.00077469   0.00124571  …  0.00143112   0.00143829 
 0.00152533   0.00148025   0.00109659     0.000531771  0.0004577  
 0.00155614   0.00152533   0.00118792     0.000611682  0.000531771
 ⋮                                     ⋱                          
 0.000954524  0.00104965   0.0014527

## log  Lambda

**Scrubbed and equivalent to R-unit test results.**

In [74]:
function getLogLambda(tau, mu, thetafxixj, Tlast=30.)
    tauLessTIndex = find(x-> x<=Tlast, tau)

    sumLog = 0
    for i = tauLessTIndex
        sumTauJLessTauI = 0
        tauJLessTauIIndex = find(x-> x < tau[i], tau)
        for j = tauJLessTauIIndex
            sumTauJLessTauI = sumTauJLessTauI + thetafxixj[i,j]
        end
        sumLog = sumLog + log(mu + sumTauJLessTauI)
    end
    sumLog
end

getLogLambda (generic function with 4 methods)

In [94]:
getLogLambda(t, u, thetafxixj, tl)

15.49438230307215

## $\mu$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [211]:
function llRatio_mu(mu, muStar, tau, thetafxixj; 
                        Tlast=30.0, mu_shape=0.7, mu_rate=0.004)
    muPrior = Gamma(mu_shape, 1./mu_rate)

    tauLessTIndex = find(x-> x<=Tlast, tau)
    tauGreaterT = count(x-> x>Tlast, tau)

    dmu = mu - muStar

    dllTerm1 = sum(tau[tauLessTIndex]*dmu)
    dllTerm2 = tauGreaterT * Tlast * dmu

    dllTerm3a = getLogLambda(tau, muStar, thetafxixj, Tlast)
    dllTerm3b = getLogLambda(tau, mu, thetafxixj, Tlast)
    dllTerm3 = dllTerm3a - dllTerm3b

    logPriorStar = log(pdf(muPrior, muStar))
    logPrior = log(pdf(muPrior, mu))

    dll = dllTerm1 + dllTerm2 + dllTerm3 + logPriorStar - logPrior
    dll
end

llRatio_mu (generic function with 2 methods)

In [290]:
llRatio_mu(u, 0.0028, t, thetafxixj, Tlast=tl)

7146.889132721996

R Value: -0.9685811338229993

## $\sigma$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [299]:
sStar=0.5;
thetafxixjStar = getThetafxixj(th, sStar, dxixj);
dthetafxixj = thetafxixjStar-thetafxixj;

In [300]:
function getSumLessT_sigma(dthetafxixj, mu, tau, Tlast=30.0)
    tauLessTIndex = find(x -> x<=Tlast, tau)

    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            #reversing i & j to make overall term positive
            tmp = (tau[j] - tau[i])*dthetafxixj[i,j]
            sum_ij = sum_ij + tmp
        end
        sumLessT = sumLessT + sum_ij
    end
    sumLessT
end

getSumLessT_sigma (generic function with 2 methods)

In [301]:
getSumLessT_sigma(dthetafxixj, u, t, tl)

-0.022907866738683627

R Value: -37.07198157467801

In [302]:
function getSumGreaterT_sigma(dthetafxixj, mu, tau, Tlast=30.)

    tauGreaterTIndex = find(x -> x>Tlast, tau)

    sumGreaterT = 0
    for i=tauGreaterTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            #reversing j & T to make overall term positive
            sum_ij = sum_ij +
                (tau[j]-Tlast)*dthetafxixj[i,j]
        end
        sumGreaterT = sumGreaterT + sum_ij
    end
    sumGreaterT
end

getSumGreaterT_sigma (generic function with 2 methods)

In [303]:
getSumGreaterT_sigma(dthetafxixj, u, t, tl)

-0.06230889750745333

R Value: 0.12761710295072787

In [304]:
function llRatio_sigma(sigma, sigmaStar, thetafxixj, 
        mu, tau; Tlast=30.0, sigma_shape=0.5, sigma_scale=100.)
    
    thetafxixjStar = getThetafxixj(th, sStar, dxixj);
    dthetafxixj = thetafxixjStar - thetafxixj
    sigmaPrior = Gamma(sigma_shape, sigma_scale)

    dllTerm1 = getSumLessT_sigma(dthetafxixj, mu, tau, Tlast)

    dllTerm2 = getSumGreaterT_sigma(dthetafxixj, mu, tau, Tlast)

    dllTerm3 = getLogLambda(tau, mu, thetafxixjStar, Tlast) -
        getLogLambda(tau, mu, thetafxixj, Tlast)

    logPriorStar = log(pdf(sigmaPrior, sigmaStar))
    logPrior = log(pdf(sigmaPrior, sigma))

    dll = dllTerm1 + dllTerm2 + dllTerm3 + logPriorStar - logPrior

    dll
end

llRatio_sigma (generic function with 2 methods)

In [305]:
llRatio_sigma(s, sStar, thetafxixj, u, t, Tlast=tl)

1.5076654030232408

R Value: -29.676456732636904 (*before log(prior) correction*)

# $\theta$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [331]:
thStar = 0.5;
thetafxixj = getThetafxixj(th, s, dxixj);
thetafxixjStar = getThetafxixj(thStar, s, dxixj);
dthetafxixj = thetafxixj - thetafxixjStar;

In [332]:
function getSumLessT_theta(theta, thetaStar, 
        thetafxixj, thetafxixjStar, dthetafxixj,
        mu, tau, sigma, dxixj, Tlast=30.0)
    
    tauLessTIndex = find(x -> x<=Tlast, tau)

    sumLessT = 0
    for i = tauLessTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j = tauJLessTauIIndex
            sum_ij = sum_ij +
                (tau[i] - tau[j])*dthetafxixj[i,j]
        end
        sumLessT = sumLessT + sum_ij
    end
    sumLessT
end

getSumLessT_theta (generic function with 4 methods)

In [336]:
getSumLessT_theta(th, 0.1, thetafxixj, thetafxixjStar, dthetafxixj, u, t, s, dxixj, tl)

0.016834894020290453

R Value: 0.893519762891437

In [334]:
function getSumGreaterT_theta(theta, thetaStar, 
        thetafxixj, thetafxixjStar, dthetafxixj,
        mu, tau, sigma, dxixj, Tlast=30.0)
    
    tauGreaterTIndex = find(x -> x>Tlast, tau)

    sumGreaterT = 0
    for i=tauGreaterTIndex
        sum_ij = 0
        tauJLessTauIIndex = find(x-> x<tau[i], tau)
        for j=tauJLessTauIIndex
            sum_ij = sum_ij + (Tlast - tau[j])*dthetafxixj[i,j]
        end
        sumGreaterT = sumGreaterT + sum_ij
    end
    sumGreaterT
end

getSumGreaterT_theta (generic function with 4 methods)

In [337]:
getSumGreaterT_theta(th, 0.1, thetafxixj, thetafxixjStar, dthetafxixj, u, t, s, dxixj, tl)

0.011924109849970606

R Value: 0.2233632497737195

In [361]:
function llRatio_theta(theta, thetaStar, mu, tau, sigma, dxixj; 
                         Tlast=30.0, theta_shape=0.8, theta_rate=10.0)

    thetafxixj = getThetafxixj(theta, sigma, dxixj)
    thetafxixjStar = getThetafxixj(thetaStar, sigma, dxixj)
    dthetafxixj = thetafxixj - thetafxixjStar
    
    thetaPrior = Gamma(theta_shape, 1./theta_rate)
    
    dllTerm1 = getSumLessT_theta(theta, thetaStar, 
        thetafxixj, thetafxixjStar, dthetafxixj, 
        mu, tau, sigma, dxixj, Tlast)

    dllTerm2 = getSumGreaterT_theta(theta, thetaStar, 
        thetafxixj, thetafxixjStar, dthetafxixj, 
        mu, tau, sigma, dxixj, Tlast)

    dllTerm3 = getLogLambda(tau, mu, thetafxixjStar, Tlast) -
                    getLogLambda(tau, mu, thetafxixj, Tlast)

    logPriorStar = log(pdf(thetaPrior, thetaStar))
    logPrior = log(pdf(thetaPrior, theta))

    dll = dllTerm1 + dllTerm2 + dllTerm3 + logPriorStar - logPrior
    dll
end

llRatio_theta (generic function with 1 method)

In [380]:
llRatio_theta(th, 0.01, u, t, s, dxixj, Tlast=10)

10.846937607399619

R Value: -1.7516207513257855

# $\tau$ log-likelihood Ratio

**Scrubbed and equivalent to R-unit test results.**

In [384]:
function getMinSum(iStar, tauiStar, tau, thetafxixj)
    ti = tau[iStar]
    minTau = min(tauiStar, ti)

    jLessMin = find(x -> x<minTau, tau)
    minSum = 0
    for j = jLessMin
        minSum = minSum + thetafxixj[iStar,j]
    end
    minSum = (ti - tauiStar)*minSum
    minSum
end

getMinSum (generic function with 2 methods)

In [385]:
getMinSum(1, t[1]+0.5, t, thetafxixj)

-0.0020571749568161143

R Value: -0.06382284020653121

In [386]:
function getMaxSum(iStar, tauiStar, tau, thetafxixj)
    ti = tau[iStar]
    maxTau = max(tauiStar, ti)

    jGreaterMax = find(x -> x>maxTau, tau)
    maxSum = 0
    for j = jGreaterMax
        maxSum = maxSum + thetafxixj[iStar,j]
    end
    maxSum = (tauiStar - ti)*maxSum
    maxSum
end

getMaxSum (generic function with 1 method)

In [387]:
getMaxSum(2, t[2]-0.5, t, thetafxixj)

-0.0007918057725081853

R Value: -0.031913804524571286

In [389]:
function getSumIntervalBelowTauStar(iStar, tauiStar, tau, thetafxixj)

    ti = tau[iStar]

    jInBetween = find(x -> ((x>ti) & (x<tauiStar)), tau)
    sumBelow = 0
    for j = jInBetween
        sumBelow = sumBelow + (2*tau[j] - ti - tauiStar)*thetafxixj[iStar,j]
    end
  sumBelow
end

getSumIntervalBelowTauStar (generic function with 1 method)

In [396]:
getSumIntervalBelowTauStar(2, 21., t, thetafxixj)

0.007918057725081853

R Value: 0.1914828271474277

In [397]:
function getSumIntervalAboveTauStar(iStar, tauiStar, tau, thetafxixj)
    ti = tau[iStar]

    jInBetween = find(x -> ((x>tauiStar) & (x<ti)), tau)
    sumAbove = 0
    for j = jInBetween
            sumAbove = sumAbove + (ti + tauiStar - 2*tau[j])*thetafxixj[iStar,j]
    end
    sumAbove
end

getSumIntervalAboveTauStar (generic function with 1 method)

In [398]:
getSumIntervalAboveTauStar(1, 8., t, thetafxixj)

0.01063499063708131

R Value: 0.12765521809828514

In [409]:
function llRatio_tau(tau, tauiStar, iStar, mu, thetafxixj, lastLogLambda; Tlast=30.0)
    
    N = length(tau)

    dllTerm1 = mu*(tau[iStar] - tauiStar)
    dllTerm2 = getMinSum(iStar, tauiStar, tau, thetafxixj)
    dllTerm3 = getMaxSum(iStar, tauiStar, tau, thetafxixj)
    dllTerm4 = getSumIntervalBelowTauStar(iStar, tauiStar, tau, thetafxixj)
    dllTerm5 = getSumIntervalAboveTauStar(iStar, tauiStar, tau, thetafxixj)

    tauStar = copy(tau)
    tauStar[iStar] = tauiStar
    dllTerm6 = getLogLambda(tauStar, theta, mu, sigma, dst, Tlast) - lastLogLambda

    dll = dllTerm1 + dllTerm2 + dllTerm3 + dllTerm4 + dllTerm5 + dllTerm6
    dll
end

llRatio_tau (generic function with 1 method)

In [420]:
lll = getLogLambda(t, u, thetafxixj, tl)
llRatio_tau(t, t[1]-4, 1, u, thetafxixj, lll, Tlast=tl)

699.9835179289495

R Value: 0.06522284020653046